![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/ml-frameworks/fastai/train-with-custom-docker/fastai-with-custom-docker.png)

# Train a model using a custom Docker image and Darknet

In this tutorial, learn how to use a custom Docker image when training models with Azure Machine Learning and leverage the Darknet framework.

## Set up the experiment
This section sets up the training experiment by initializing a workspace, creating an experiment, and uploading the training data and training scripts.

### Initialize a workspace
The Azure Machine Learning workspace is the top-level resource for the service. It provides you with a centralized place to work with all the artifacts you create. In the Python SDK, you can access the workspace artifacts by creating a `workspace` object.

Create a workspace object from the `config.json` file.

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()

### Prepare scripts
Create a directory titled `darknet-scripts`.

In [ ]:
import os
os.makedirs('darknet-scripts', exist_ok=True)

Then run the cell below to create the training script train.py in the directory.

In [ ]:
%%writefile darknet-scripts/train.py
"""
Azure ML training script for Darknet object detection experiment
"""
import os
import requests
import subprocess
import shutil


# Test container
def greeting():
    print("Welcome to darknet container!")

greeting()

print("Getting the test image...")
# Get a test image
url = "https://raw.githubusercontent.com/AlexeyAB/darknet/master/data/giraffe.jpg"
response = requests.get(url)
if response.status_code == 200:
    with open("giraffe.jpg", "wb") as f:
        f.write(response.content)

# Make coco.data file
coco_data = """
classes= 80
train  = train.txt
valid  = val.txt
names = coco.names
backup = backup/
eval=coco
"""
print("Making coco.data")
with open("coco.data", "w") as f:
    f.write(coco_data)

print("Getting coco.names...")
# Get the names file
url = "https://raw.githubusercontent.com/AlexeyAB/darknet/master/cfg/coco.names"
response = requests.get(url)
if response.status_code == 200:
    with open("coco.names", "wb") as f:
        f.write(response.content)

print("Getting the weights file, yolov4.weights...")
# Get the weights file
url = "https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights"
response = requests.get(url)
if response.status_code == 200:
    with open("yolov4.weights", "wb") as f:
        f.write(response.content)

print("Getting the config file...")
# Get the config file
url = "https://raw.githubusercontent.com/AlexeyAB/darknet/master/cfg/yolov4.cfg"
response = requests.get(url)
if response.status_code == 200:
    with open("yolov4.cfg", "wb") as f:
        f.write(response.content)

# What is our current working directory?
print("Current working directory: {}".format(os.getcwd()))
print("Contents of directory: ")
os.system("ls")

# Predict with darknet
print("Running darknet detector test!")
os.system("darknet detector test coco.data yolov4.cfg yolov4.weights -thresh 0.25 giraffe.jpg -ext_output")
os.system("ls")

if os.path.exists("predictions.jpg"):
    shutil.copyfile("predictions.jpg", "outputs/predictions.jpg")
if os.path.exists("predictions.png"):
    shutil.copyfile("predictions.png", "outputs/predictions.png")


print("Return value: {}".format(retval))

### Define your environment
Create an environment object and enable Docker.

In [ ]:
from azureml.core import Environment

darknet_env = Environment("darknet")
darknet_env.docker.enabled = True

This specified base image supports the darknet framework which allows for object detection deep learning capabilities. For more information, see the [darknet GitHub repo](https://github.com/AlexeyAB/darknet). 

When you are using your custom Docker image, you might already have your Python environment properly set up. In that case, set the `user_managed_dependencies` flag to True in order to leverage your custom image's built-in python environment.

In [ ]:
darknet_env.docker.base_image = None
darknet_env.docker.base_dockerfile = "./Dockerfile"
# darknet_env.python.user_managed_dependencies = True

To use an image from a private container registry that is not in your workspace, you must use `docker.base_image_registry` to specify the address of the repository as well as a username and password.

```python
fastai_env.docker.base_image_registry.address = "myregistry.azurecr.io"
fastai_env.docker.base_image_registry.username = "username"
fastai_env.docker.base_image_registry.password = "password"
```

It is also possible to use a custom Dockerfile. Use this approach if you need to install non-Python packages as dependencies and remember to set the base image to None. 

Specify docker steps as a string:
```python 
dockerfile = r""" \
FROM mcr.microsoft.com/azureml/base:intelmpi2018.3-ubuntu16.04
RUN echo "Hello from custom container!" \
"""
```
Set base image to None, because the image is defined by dockerfile:
```python
fastai_env.docker.base_image = None \
fastai_env.docker.base_dockerfile = dockerfile
```
Alternatively, load the string from a file:
```python
fastai_env.docker.base_image = None \
fastai_env.docker.base_dockerfile = "./Dockerfile"
```

### Create or attach existing AmlCompute
You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for training your model. In this tutorial, you create `AmlCompute` as your training compute resource.

**Creation of AmlCompute takes approximately 5 minutes.** If the AmlCompute with that name is already in your workspace this code will skip the creation process.

As with other Azure services, there are limits on certain resources (e.g. AmlCompute) associated with the Azure Machine Learning service. Please read [this article](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-manage-quotas) on the default limits and how to request more quota.

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "gpuforkeras"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6',
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current AmlCompute
print(compute_target.get_status().serialize())

### Create a ScriptRunConfig
This ScriptRunConfig will configure your job for execution on the desired compute target.

In [ ]:
from azureml.core import ScriptRunConfig

darknet_config = ScriptRunConfig(source_directory='darknet-scripts',
                                script='train.py',
                                compute_target=compute_target,
                                environment=darknet_env)

### Submit your run
When a training run is submitted using a ScriptRunConfig object, the submit method returns an object of type ScriptRun. The returned ScriptRun object gives you programmatic access to information about the training run. 

In [ ]:
from azureml.core import Experiment

run = Experiment(ws,'darknet-custom-image').submit(darknet_config)
run.wait_for_completion(show_output=True)